In this notebook, we explain four strategies based on the power of each user. We will rank the users from the strongest to the weakest and then we will allocate them to one of the two groups according to specific strageies explained below.

In [1]:
N = 4 # number of antennas in the BS
K = 10 # number of users (K >= N)
X0 = 0.500 # parameter of the uniform distribution of the user on the x-axis
Y0 = 0.500 # parameter of the uniform distribution of the user on the y-axis
nsize = 10000 # size of the dataset

In [2]:
from itertools import product, combinations
import numpy as np
from numpy.linalg import inv
import h5py
import pandas as pd
from PIL import Image
from numpy import linalg as LA
import os
from sklearn.model_selection import train_test_split
from PIL import Image
from matplotlib.pyplot import imshow
import seaborn as sns
from sklearn import metrics
from IPython.display import clear_output
%matplotlib inline
np.random.seed(42) # fix the seed for reproducibility

In [3]:
def hermetian(H):
    return H.conj().T

In [4]:
def SNR1(H, P, sigma, i):
    K = H.shape[1]
    N = H.shape[0]
    P_i= P[i]
    h_i = H[:,i][:, np.newaxis]
    P = np.delete(P, i)
    set_minus_i = list(range(K))
    set_minus_i.pop(i)
    SNR = P_i*hermetian(h_i).dot(inv(H[:,set_minus_i].dot(np.diag(P)).dot(hermetian(H[:,set_minus_i])) + sigma**2*np.identity(N))).dot(h_i)
    return SNR.real

In [5]:
def SNR2(H, P, sigma, i, set_I1):
    K = H.shape[1]
    N = H.shape[0]
    P_i= P[i]
    set_0 = set_I1 + [i]
    h_i = H[:,i][:, np.newaxis]
    P = np.delete(P, set_0)
    set_minus_i = list(range(K))
    set_minus_i = [j for j in set_minus_i if j not in set_0]
    SNR = P_i*hermetian(h_i).dot(inv(H[:,set_minus_i].dot(np.diag(P)).dot(hermetian(H[:,set_minus_i])) + sigma**2*np.identity(N))).dot(h_i)
    return SNR.real

In [6]:
def sum_rate(df):
    sigma = 1
    with h5py.File('test_Data/' + df['H'], 'r') as hf:
        H = hf[df['H'][:-3]][:]
    with h5py.File('test_Data/' + df['P'], 'r') as hf:
        P = hf[df['P'][:-3]][:]
    P_diag = P.diagonal()
    N = H.shape[0]
    K = H.shape[1]
    cols = ['User'+str(i+1) for i in range(K)]
    conf = df[cols].values
    set_I1 = np.where(conf == 0)[0].tolist()
    set_I2 = np.where(conf == 1)[0].tolist()
    SNR_1 = [SNR1(H, P_diag, sigma, s) for s in set_I1] # SNR_i^(1) for i in I_1
    SNR_2 = [SNR2(H, P_diag, sigma, s, set_I1) for s in set_I2] # SNR_i^(2) for i in I_2
    R_1 = np.sum(np.log(1+np.array(SNR_1)))
    R_2 = np.sum(np.log(1+np.array(SNR_2)))
    return R_1 + R_2

In [7]:
cols = ['User' + str(i+1) for i in range(K)]

In [8]:
test_df = pd.read_csv('test_data.csv')

# First strategy

In this first strategy, we will put the first strongest users into group 1 and the weakest into group 2.

In [9]:
def snr1_label(df):
    N = 4
    K = 10
    with h5py.File('test_Data/' + df['H'], 'r') as hf:
        H = hf[df['H'][:-3]][:]
    with h5py.File('test_Data/' + df['P'], 'r') as hf:
        P = hf[df['P'][:-3]][:]
    
    P = np.diag(P)
    power = []
    for i in range(K):
        h_i = H[:,i][:, np.newaxis]
        P_i = P[i]
        power.append(LA.norm(h_i)**2*P_i)
        
    s = sorted(range(len(power)), key=lambda k: power[k])
                           
    set0 = [s[i] for i in range(int(len(s)/2))]
    set1 = [s[i] for i in range(int(len(s)/2), len(s))]
    # transform it to one vector
    labels = np.zeros(K)
    for i in range(K):
        if  i in set0:
            labels[i] = 0
        else:
            labels[i] = 1

    return list(labels)

In [10]:
column_names=['User' + str(i+1) for i in range(K)]
snr1_df = pd.DataFrame(columns = column_names)

In [11]:
x= test_df.apply(snr1_label, axis = 1)

In [12]:
for j in range(2000):
    snr1_df.loc[j] = x[j]

In [13]:
snr1_df['H'] = test_df['H']
snr1_df['P'] = test_df['P']

In [14]:
test_df['sum_rate'] = test_df.apply(sum_rate, axis = 1)

In [15]:
snr1_df.head()

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,H,P
0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,H_1.h5,P_1.h5
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,H_2.h5,P_2.h5
2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,H_3.h5,P_3.h5
3,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,H_4.h5,P_4.h5
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,H_5.h5,P_5.h5


In [16]:
snr1_df['sum_rate'] = snr1_df.apply(sum_rate, axis = 1)

In [17]:
snr1_df.head()

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,H,P,sum_rate
0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,H_1.h5,P_1.h5,23.516408
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,H_2.h5,P_2.h5,18.734753
2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,H_3.h5,P_3.h5,11.070492
3,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,H_4.h5,P_4.h5,13.418563
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,H_5.h5,P_5.h5,10.378225


In [18]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(test_df['sum_rate'],snr1_df['sum_rate']))
mape = np.mean(np.abs((test_df['sum_rate'] - snr1_df['sum_rate']) / test_df['sum_rate'])) * 100
print('RMSE of the sum rate on the test set is:', round(rmse,2))
print('MAPE of the sum rate on the test set is:', round(mape,2))

RMSE of the sum rate on the test set is: 10.78
MAPE of the sum rate on the test set is: 39.19


## Second Strategy 

In the second strategy, we will put the weakest users into the first group and the strongest into the second group.

In [19]:
def snr2_label(df):
    N = 4
    K = 10
    with h5py.File('test_Data/' + df['H'], 'r') as hf:
        H = hf[df['H'][:-3]][:]
    with h5py.File('test_Data/' + df['P'], 'r') as hf:
        P = hf[df['P'][:-3]][:]
    
    P = np.diag(P)
    power = []
    for i in range(K):
        h_i = H[:,i][:, np.newaxis]
        P_i = P[i]
        power.append(LA.norm(h_i)**2*P_i)
        
    s = sorted(range(len(power)), key=lambda k: power[k])
                           
    set1 = [s[i] for i in range(int(len(s)/2))]
    set0 = [s[i] for i in range(int(len(s)/2), len(s))]
    # transform it to one vector
    labels = np.zeros(K)
    for i in range(K):
        if  i in set0:
            labels[i] = 0
        else:
            labels[i] = 1

    return list(labels)

In [20]:
column_names=['User' + str(i+1) for i in range(K)]
snr2_df = pd.DataFrame(columns = column_names)

In [21]:
x= test_df.apply(snr2_label, axis = 1)

In [22]:
for j in range(2000):
    snr2_df.loc[j] = x[j]

In [23]:
snr2_df['H'] = test_df['H']
snr2_df['P'] = test_df['P']

In [24]:
snr2_df.head()

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,H,P
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,H_1.h5,P_1.h5
1,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,H_2.h5,P_2.h5
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,H_3.h5,P_3.h5
3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,H_4.h5,P_4.h5
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,H_5.h5,P_5.h5


In [25]:
snr2_df['sum_rate'] = snr2_df.apply(sum_rate, axis = 1)

In [26]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(test_df['sum_rate'],snr2_df['sum_rate']))
mape = np.mean(np.abs((test_df['sum_rate'] - snr2_df['sum_rate']) / test_df['sum_rate'])) * 100
print('RMSE of the sum rate on the test set is:', round(rmse,2))
print('MAPE of the sum rate on the test set is:', round(mape,2))

RMSE of the sum rate on the test set is: 9.12
MAPE of the sum rate on the test set is: 33.38


## Third Strategy

In this strategy, we will put $U_{odd}$ (set of odd indices) into group 1 and $U_{even}$ (set of even indices) into group 2.

In [27]:
def snr3_label(df):
    N = 4
    K = 10
    with h5py.File('test_Data/' + df['H'], 'r') as hf:
        H = hf[df['H'][:-3]][:]
    with h5py.File('test_Data/' + df['P'], 'r') as hf:
        P = hf[df['P'][:-3]][:]
    
    P = np.diag(P)
    power = []
    for i in range(K):
        h_i = H[:,i][:, np.newaxis]
        P_i = P[i]
        power.append(LA.norm(h_i)**2*P_i)
        
    s = sorted(range(len(power)), key=lambda k: power[k])
                           
    set0 = s[0:][::2] # even
    set1 = s[1:][::2] # odd
    # transform it to one vector
    labels = np.zeros(K)
    for i in range(K):
        if  i in set0:
            labels[i] = 0
        else:
            labels[i] = 1

    return list(labels)

In [28]:
column_names=['User' + str(i+1) for i in range(K)]
snr3_df = pd.DataFrame(columns = column_names)

In [29]:
x= test_df.apply(snr3_label, axis = 1)

In [30]:
for j in range(2000):
    snr3_df.loc[j] = x[j]

In [31]:
snr3_df['H'] = test_df['H']
snr3_df['P'] = test_df['P'] 

In [32]:
snr3_df.head()

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,H,P
0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,H_1.h5,P_1.h5
1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,H_2.h5,P_2.h5
2,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,H_3.h5,P_3.h5
3,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,H_4.h5,P_4.h5
4,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,H_5.h5,P_5.h5


In [ ]:
snr3_df['sum_rate'] = snr3_df.apply(sum_rate, axis = 1)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(test_df['sum_rate'],snr3_df['sum_rate']))
mape = np.mean(np.abs((test_df['sum_rate'] - snr3_df['sum_rate']) / test_df['sum_rate'])) * 100
print('RMSE of the sum rate on the test set is:', round(rmse,2))
print('MAPE of the sum rate on the test set is:', round(mape,2))

## Fourth Strategy

In this strategy, we will put $U_{eve}$ (set of odd indices) into group 1 and $U_{even}$ (set of even indices) into group 2.

In [52]:
def snr4_label(df):
    N = 4
    K = 10
    with h5py.File('test_Data/' + df['H'], 'r') as hf:
        H = hf[df['H'][:-3]][:]
    with h5py.File('test_Data/' + df['P'], 'r') as hf:
        P = hf[df['P'][:-3]][:]
    
    P = np.diag(P)
    power = []
    for i in range(K):
        h_i = H[:,i][:, np.newaxis]
        P_i = P[i]
        power.append(LA.norm(h_i)**2*P_i)
        
    s = sorted(range(len(power)), key=lambda k: power[k])
                           
    set1 = s[0:][::2] # even
    set0 = s[1:][::2] # odd
    # transform it to one vector
    labels = np.zeros(K)
    for i in range(K):
        if  i in set0:
            labels[i] = 0
        else:
            labels[i] = 1

    return list(labels)

In [53]:
column_names=['User' + str(i+1) for i in range(K)]
snr4_df = pd.DataFrame(columns = column_names)

In [54]:
x= test_df.apply(snr4_label, axis = 1)

In [55]:
for j in range(2000):
    snr4_df.loc[j] = x[j]

In [56]:
snr4_df['H'] = test_df['H']
snr4_df['P'] = test_df['P']

In [57]:
snr3_df.head()

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,H,P,sum_rate
0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,H_1.h5,P_1.h5,19.136739
1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,H_2.h5,P_2.h5,23.254066
2,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,H_3.h5,P_3.h5,14.389836
3,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,H_4.h5,P_4.h5,9.007951
4,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,H_5.h5,P_5.h5,9.732334


In [58]:
snr4_df['sum_rate'] = snr4_df.apply(sum_rate, axis = 1)

In [59]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(test_df['sum_rate'],snr4_df['sum_rate']))
mape = np.mean(np.abs((test_df['sum_rate'] - snr4_df['sum_rate']) / test_df['sum_rate'])) * 100
print('RMSE of the sum rate on the test set is:', round(rmse,2))
print('MAPE of the sum rate on the test set is:', round(mape,2))

RMSE of the sum rate on the test set is: 8.24
MAPE of the sum rate on the test set is: 29.94
